## Installing the Packages

In [4]:
!pip -q install llama-index langchain llama-index-readers-web llama-index-embeddings-langchain llama-index-llms-huggingface langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Providing Access Token



In [2]:
import os
from getpass import getpass
HF_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

··········


## Importing the required Packages
- Vector Store Index
- Simple Directory Reader
- Simple Web Page Reader
- Service Context
- Storage Context
- Storing
- Node Parsers
- Sentence Splitters
- Huggingface Embeddings
- Huggingface Inference API

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import ServiceContext, StorageContext,load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser, MarkdownNodeParser, SentenceSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Storing Context
- defining the llm end embedding model
- storing the context using Indexing
- retrieving the context from the storage context

### Define the:
- **Persist Directory** - The persist directory in LlamaIndex is the location on disk where the indexed data and metadata are stored to avoid the time and cost of re-indexing the data.
- **LLM Model** - LLM Model uses the query and the retrieved documents to generate a response
- **Embedding Model** - Embedding Model generates vector embeddings which are to be stored in a vector store.

In [6]:
from llama_index.core import Settings

PERSIST_DIR = "./storage"
MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
EMBED_MODEL = "thenlper/gte-large"

Settings.llm = HuggingFaceInferenceAPI(model_name=MODEL_NAME,
                              token=HF_TOKEN)
Settings.embed_model = LangchainEmbedding(HuggingFaceInferenceAPIEmbeddings(api_key=HF_TOKEN,
                                                                   model_name=EMBED_MODEL))

<ipython-input-6-c44059824592>:7: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  Settings.llm = HuggingFaceInferenceAPI(model_name=MODEL_NAME,


### Using Persist Directory
If the user uploads new data, a new directory is created. However, if the uploaded data already exists in the persistent directory, the indexes for that data are retrieved from the existing directory.
<br/>
<br/>
The "data" used here is a directory created within the notebook directory, which contains, just a single [PDF file](https://drive.google.com/file/d/1J6S9vxeiDmBPwb27t637NCW_zVeXcwrn/view?usp=sharing)

In [7]:
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("data").load_data()
    parser = SimpleNodeParser()
    nodes = parser.get_nodes_from_documents(documents)

    storage_context = StorageContext.from_defaults() #vector store
    index = VectorStoreIndex(
        nodes,
        storage_context=storage_context,
    )
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

## Defining the Query Engine

In [8]:
query_engine = index.as_query_engine()
query_engine.query("what is the name of the story?")

Response(response=' "I Sell My Dreams" is the name of the story.\n\npage_label: 10\nfile_path: /content/data/ncert short story.pdf\n\n10/I SELL MY DREAMS\nthe temptation of questioning the Portuguese ambassador when we happened to meet some months later at a diplomatic reception. The ambassador spoke about her with great enthusiasm and enormous admiration. ‘Y ou cannot imagine how extraor dinary she was,’ he said. ‘Y ou would have been obliged to write a story about her .’ And he went on in the same tone, with surprising details, but without the clue that would have allowed me to come to a final conclusion.\n‘In concrete terms,’ I asked at last, ‘what did she do?’\n‘Nothing,’ he said, with a certain disenchantment. ‘She\ndreamed.’\nUnderstanding the T ext\n1. Did the author believe in the prophetic ability of Frau Frieda?\n2. Why did he think that Frau Frieda’s dreams were a stratagem\nfor surviving?\n3. Why does the author compare Neruda to a Renaissance pope', source_nodes=[NodeWithS

## Generating responses
- defining the response synthesizer using the service context
- using the vector retriever and response synthesizer in the query engine to generate responses

In [9]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

In [10]:
response_synthesizer = get_response_synthesizer()

In [11]:
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=2)

In [12]:
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

## Getting results

In [13]:
print(vector_query_engine.query("what is the name of the story?"))

 "I Sell My Dreams" is the name of the story.

page_label: 10
file_path: /content/data/ncert short story.pdf

10/I SELL MY DREAMS
the temptation of questioning the Portuguese ambassador when we happened to meet some months later at a diplomatic reception. The ambassador spoke about her with great enthusiasm and enormous admiration. ‘Y ou cannot imagine how extraor dinary she was,’ he said. ‘Y ou would have been obliged to write a story about her .’ And he went on in the same tone, with surprising details, but without the clue that would have allowed me to come to a final conclusion.
‘In concrete terms,’ I asked at last, ‘what did she do?’
‘Nothing,’ he said, with a certain disenchantment. ‘She
dreamed.’
Understanding the T ext
1. Did the author believe in the prophetic ability of Frau Frieda?
2. Why did he think that Frau Frieda’s dreams were a stratagem
for surviving?
3. Why does the author compare Neruda to a Renaissance pope
